# Aprendizagem supervisionada: classificação

Este projeto tem como objetivo desenvolver um aplicativo de machine learning para prever a tendência de uma pessoa desenvolver algum tipo de 
doença cardíaca com base em alguns fatores clínicos e laboratoriais.

Os dados foram extraídos do Kaggle: https://www.kaggle.com/fedesoriano/heart-failure-prediction/version/1

## Pré-processamento

In [1]:
import numpy as np
import pandas as pd

In [2]:
# apesar de ter sido salvo como utf-8, neste caso (não é recomendável), poderia usar iso-8859-1 que também funciona
# df = pd.read_csv('../datasets/heart_tratado.csv', sep=';', encoding='iso-8859-1')
df = pd.read_csv('../datasets/heart_tratado.csv', sep=';', encoding='utf-8')

In [3]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289.0,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180.0,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283.0,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214.0,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195.0,0,Normal,122,N,0.0,Up,0


## Transformando variáveis categóricas nominais em variáveis categóricas ordinais

Onde a categoria for um nome, está será transformada em número, por exemplo: Sex, M ou F, para 0 e 1, ou 1 e 2...

In [7]:
df2 = pd.DataFrame.copy(df)

In [43]:
df2['Sex'].replace({'M': 0, 'F': 1}, inplace=True)
df2['ChestPainType'].replace({'TA': 0, 'ATA': 1, 'NAP': 2, 'ASY': 3}, inplace=True)
df2['RestingECG'].replace({'Normal': 0, 'ST': 1, 'LVH': 2}, inplace=True)
df2['ExerciseAngina'].replace({'N': 0, 'Y': 1}, inplace=True)
df2['ST_Slope'].replace({'Up': 0, 'Flat': 1, 'Down': 2}, inplace=True)

C:\Users\rafae\AppData\Local\Temp\ipykernel_15328\3930933072.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['RestingECG'].replace({'Normal': 0, 'ST': 1, 'LVH': 2}, inplace=True)
C:\Users\rafae\AppData\Local\Temp\ipykernel_15328\3930933072.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  

In [44]:
df2.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,1,140,289.0,0,0,172,0,0.0,0,0
1,49,1,2,160,180.0,0,0,156,0,1.0,1,1
2,37,0,1,130,283.0,0,1,98,0,0.0,0,0
3,48,1,3,138,214.0,0,0,108,1,1.5,1,1
4,54,0,2,150,195.0,0,0,122,0,0.0,0,0


## Legenda

- Age = idade
- Sex = sexo
- Chest Pain Type = tipo de dor no peito (TA = angina típica, ATA = angina atípica, NAP = dor não anginosa, ASY = assintomático)
- Resting BP = pressão sanguínea em repouso (mmHg)
- Cholesterol = colesterol sérico (mg/dl)
- Fasting BS = açúcar no sangue em jejum (mg/dl), < 120 = não-diabético, >= 120 = diabético
- Resting ECG = eletrocardiograma em repouso (Normal, ST = Anormalidade da onda ST-T, LVH = Hipertrofia ventricular esquerda)
- Max HR = frequência cardíaca máxima
- Exercise Angina = Angina induzida por exercício
- Old Peak = depressão de ST induzida por exercício em relação ao repouso
- ST_Slope = inclinação do segmento ST (Up, Flat, Down)
- Heart Disease = Doença cardíaca (possui / não possui)

## Atributos previsores e alvo

In [ ]:
# a função iloc(), busca considerando um índice
# Neste caso, será criado uma matrix com os valores, até a 10a coluna
previsores = df2.iloc[:, 0:11].values
# warning
# We recommend using DataFrame.to_numpy instead.
# Only the values in the DataFrame will be returned, the axes labels will be removed.
# previsores = df2.to_numpy() # Assim fica com uma coluna a mais -> previsores.shape = (917, 12)
# Corrigindo:
# previsores = df2.to_numpy()[:, 0:11]


In [46]:
previsores.shape

(917, 11)

In [39]:
print(type(previsores))
print(previsores)

<class 'numpy.ndarray'>
[[40 0 1 ... 0 0.0 0]
 [49 1 2 ... 0 1.0 1]
 [37 0 1 ... 0 0.0 0]
 ...
 [57 0 3 ... 1 1.2 1]
 [57 1 1 ... 0 0.0 1]
 [38 0 2 ... 0 0.0 0]]


In [ ]:
# só irá pegar a coluna 11, gerando um array com seus valores
alvo = df2.iloc[:, 11].values

In [14]:
alvo.shape

(917,)

In [28]:
alvo

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [17]:
df2.describe()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
count,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.00000,917.000000
mean,53.509269,0.210469,2.251908,132.540894,244.635389,0.233370,136.789531,0.404580,0.886696,0.63795,0.552890
std,9.437636,0.407864,0.931502,17.999749,53.347125,0.423206,25.467129,0.491078,1.066960,0.60727,0.497466
min,28.000000,0.000000,0.000000,80.000000,85.000000,0.000000,60.000000,0.000000,-2.600000,0.00000,0.000000
25%,47.000000,0.000000,2.000000,120.000000,214.000000,0.000000,120.000000,0.000000,0.000000,0.00000,0.000000
50%,54.000000,0.000000,3.000000,130.000000,244.635389,0.000000,138.000000,0.000000,0.600000,1.00000,1.000000
75%,60.000000,0.000000,3.000000,140.000000,267.000000,0.000000,156.000000,1.000000,1.500000,1.00000,1.000000
max,77.000000,1.000000,3.000000,200.000000,603.000000,1.000000,202.000000,1.000000,6.200000,2.00000,1.000000


Padronização (usa média e o desvio padrão como referência)
Normalização (usa os valores de máximo e mínimo como referência)

Será usada a padronização

In [18]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# verificando se há valores de string, porque se tiver, vai dar erro no fit_transform()
# has_string = np.any([isinstance(item, str) for item in previsores.flatten()])
# print('Contém strings?', has_string) # true

# string_values = set([item for item in previsores.flatten() if isinstance(item, str)])
# recommended way by sonar
# string_values = {item for item in previsores.flatten() if isinstance(item, str)}
# print("Valores string encontrados:", string_values)

# escalonamento dos previsores
scaler = StandardScaler()
previsores_esc = scaler.fit_transform(previsores) # ValueError: could not convert string to float: 'Normal'

In [49]:
previsores_esc

array([[-1.43220634, -0.51630861, -1.34470119, ..., -0.82431012,
        -0.83150225, -1.05109458],
       [-0.47805725,  1.9368261 , -0.27058012, ..., -0.82431012,
         0.10625149,  0.59651863],
       [-1.75025603, -0.51630861, -1.34470119, ..., -0.82431012,
        -0.83150225, -1.05109458],
       ...,
       [ 0.37007527, -0.51630861,  0.80354095, ...,  1.21313565,
         0.29380223,  0.59651863],
       [ 0.37007527,  1.9368261 , -1.34470119, ..., -0.82431012,
        -0.83150225,  0.59651863],
       [-1.64423947, -0.51630861, -0.27058012, ..., -0.82431012,
        -0.83150225, -1.05109458]], shape=(917, 11))

### Codificação de variáveis categóricas

#### LabelEncoder: transformação de variáveis categóricas em numéricas

In [50]:
from sklearn.preprocessing import LabelEncoder

In [53]:
# no caso estamos usando o dataset original tratado com os dados nominais
previsores2 = df.to_numpy()[:, 0:11]   # outra forma de fazer df2.iloc[:, 0:11].values

In [ ]:
previsores2

array([[40, 'M', 'ATA', ..., 'N', 0.0, 'Up'],
       [49, 'F', 'NAP', ..., 'N', 1.0, 'Flat'],
       [37, 'M', 'ATA', ..., 'N', 0.0, 'Up'],
       ...,
       [57, 'M', 'ASY', ..., 'Y', 1.2, 'Flat'],
       [57, 'F', 'ATA', ..., 'N', 0.0, 'Flat'],
       [38, 'M', 'NAP', ..., 'N', 0.0, 'Up']],
      shape=(917, 11), dtype=object)

#### Aplicando LabelEncoder em todas as colunas que contém strings

In [ ]:
str_cols = [col for col in df.columns if df[col].apply(type).eq(str).any()]
str_cols

['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

In [61]:
cols_with_index = {col: df.columns.get_loc(col) for col in str_cols}
cols_with_index

{'Sex': 1,
 'ChestPainType': 2,
 'RestingECG': 6,
 'ExerciseAngina': 8,
 'ST_Slope': 10}

In [69]:
print(cols_with_index['Sex'])

1


In [70]:
encoder = LabelEncoder()

for col in str_cols:
    col_index = cols_with_index[col]
    previsores2[:, col_index] = encoder.fit_transform(previsores2[:, col_index])

In [71]:
previsores2

array([[40, 1, 1, ..., 0, 0.0, 2],
       [49, 0, 2, ..., 0, 1.0, 1],
       [37, 1, 1, ..., 0, 0.0, 2],
       ...,
       [57, 1, 0, ..., 1, 1.2, 1],
       [57, 0, 1, ..., 0, 0.0, 1],
       [38, 1, 2, ..., 0, 0.0, 2]], shape=(917, 11), dtype=object)

### OneHotEncoder: criação de variáveis dummy (fictícias)

Cuidado com a multicolinearidade (variáveis altamente correlacionadas entre si).
<p>
Exemplo: "Você faz atividade física?"<br/>
A = 0 ==> não<br/>
B = 1 ==> sim, 1 ou 2 dias por semana<br/>
C = 2 ==> sim, 3 ou 4 dias por semana<br/>
D = 3 ==> sim, pelo menos 5 dias por semana
</p>

In [72]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [73]:
ColumnTransformer?

Init signature:
ColumnTransformer(
    transformers,
    *,
    remainder='drop',
    sparse_threshold=0.3,
    n_jobs=None,
    transformer_weights=None,
    verbose=False,
    verbose_feature_names_out=True,
    force_int_remainder_cols='deprecated',
)
Docstring:     
Applies transformers to columns of an array or pandas DataFrame.

This estimator allows different columns or column subsets of the input
to be transformed separately and the features generated by each transformer
will be concatenated to form a single feature space.
This is useful for heterogeneous or columnar data, to combine several
feature extraction mechanisms or transformations into a single transformer.

Read more in the :ref:`User Guide <column_transformer>`.

.. versionadded:: 0.20

Parameters
----------
transformers : list of tuples
    List of (name, transformer, columns) tuples specifying the
    transformer objects to be applied to subsets of the data.

    name : str
        Like in Pipeline and FeatureUnion

Parâmetros ColumnTransformer:
- name = nome da transformação
- transformer = tipo de estimador (OneHotEncoder)
- columns = colunas que serão transformadas
- remainder = o que acontecerá com o restante das colunas não relacionadas: 1) drop = exclui as outras, 2) passthrough = mantém (default: drop)
- sparse_threshould = parâmetro de clasificacao de matrizes esparsas. default = 0.3
- n_job = numero de trabalhos executados em paralelo. default = nenhum
- transformer_weights = definicao de pesos aos transformadores
- verbose = exibicao da execucao na tela. default false

In [ ]:
previsores3 = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [1,2,6,8,10])],remainder='passthrough').fit_transform(previsores2)

# [1,2,6,8,10] se refere à colunas a serem transformadas
# poderia usar list(cols_with_index.values()), ficando conforme abaixo:
# previsores3 = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), list(cols_with_values()))],remainder='passthrough').fit_transform(previsores2)

In [76]:
previsores3

array([[0.0, 1.0, 0.0, ..., 0, 172, 0.0],
       [1.0, 0.0, 0.0, ..., 0, 156, 1.0],
       [0.0, 1.0, 0.0, ..., 0, 98, 0.0],
       ...,
       [0.0, 1.0, 1.0, ..., 0, 115, 1.2],
       [1.0, 0.0, 0.0, ..., 0, 174, 0.0],
       [0.0, 1.0, 0.0, ..., 0, 173, 0.0]], shape=(917, 20), dtype=object)

In [77]:
# visualiza;áo em dataframe
previsores3df = pd.DataFrame(previsores3)
previsores3df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,40,140,289.0,0,172,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,49,160,180.0,0,156,1.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,37,130,283.0,0,98,0.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,48,138,214.0,0,108,1.5
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,54,150,195.0,0,122,0.0


### Escalonamento

In [78]:
from sklearn.preprocessing import StandardScaler

In [79]:
previsores3_esc = StandardScaler().fit_transform(previsores3)

In [ ]:
previsores3_esc

array([[-0.51630861,  0.51630861, -1.08542493, ..., -0.55173333,
         1.38333943, -0.83150225],
       [ 1.9368261 , -1.9368261 , -1.08542493, ..., -0.55173333,
         0.75473573,  0.10625149],
       [-0.51630861,  0.51630861, -1.08542493, ..., -0.55173333,
        -1.52395266, -0.83150225],
       ...,
       [-0.51630861,  0.51630861,  0.92129817, ..., -0.55173333,
        -0.85606123,  0.29380223],
       [ 1.9368261 , -1.9368261 , -1.08542493, ..., -0.55173333,
         1.46191489, -0.83150225],
       [-0.51630861,  0.51630861, -1.08542493, ..., -0.55173333,
         1.42262716, -0.83150225]], shape=(917, 20))

In [ ]:
previsores3df = pd.DataFrame(previsores3_esc)

In [84]:
previsores3df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.516309,0.516309,-1.085425,2.073784,-0.531524,-0.229810,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,-1.001091,1.149573,-1.432206,0.414627,0.832075,-0.551733,1.383339,-0.831502
1,1.936826,-1.936826,-1.085425,-0.482210,1.881384,-0.229810,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,0.998910,-0.869888,-0.478057,1.526360,-1.212261,-0.551733,0.754736,0.106251
2,-0.516309,0.516309,-1.085425,2.073784,-0.531524,-0.229810,-0.507826,-1.226974,2.037569,0.824310,-0.824310,-0.271607,-1.001091,1.149573,-1.750256,-0.141240,0.719543,-0.551733,-1.523953,-0.831502
3,1.936826,-1.936826,0.921298,-0.482210,-0.531524,-0.229810,-0.507826,0.815013,-0.490781,-1.213136,1.213136,-0.271607,0.998910,-0.869888,-0.584074,0.303453,-0.574578,-0.551733,-1.131075,0.575128
4,-0.516309,0.516309,-1.085425,-0.482210,1.881384,-0.229810,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,-1.001091,1.149573,0.052026,0.970493,-0.930931,-0.551733,-0.581047,-0.831502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,-0.516309,0.516309,-1.085425,-0.482210,-0.531524,4.351412,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,0.998910,-0.869888,-0.902124,-1.252973,0.363191,-0.551733,-0.188170,0.293802
913,-0.516309,0.516309,0.921298,-0.482210,-0.531524,-0.229810,-0.507826,0.815013,-0.490781,0.824310,-0.824310,-0.271607,0.998910,-0.869888,1.536257,0.636973,-0.968441,1.812470,0.165420,2.356860
914,-0.516309,0.516309,0.921298,-0.482210,-0.531524,-0.229810,-0.507826,0.815013,-0.490781,-1.213136,1.213136,-0.271607,0.998910,-0.869888,0.370075,-0.141240,-2.131275,-0.551733,-0.856061,0.293802
915,1.936826,-1.936826,-1.085425,2.073784,-0.531524,-0.229810,1.969177,-1.226974,-0.490781,0.824310,-0.824310,-0.271607,0.998910,-0.869888,0.370075,-0.141240,-0.161960,-0.551733,1.461915,-0.831502


In [ ]:
previsores3df.describe()
# Conforme pode ser visto com esse comando, o objetivo do escalonamento é colocar a média (mean), próximo de 0 e o desvio padrão (std), próximo de 1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
count,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,917.000000,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,917.000000,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02
mean,1.084798e-16,-1.472226e-16,1.937139e-17,-3.874279e-17,3.874279e-17,6.973702e-17,0.000000,-9.298269e-17,1.549712e-17,-4.261707e-17,4.261707e-17,8.523413e-17,0.000000,-3.874279e-17,1.859654e-16,7.884157e-16,3.014189e-15,-1.549712e-17,-5.114048e-16,-1.859654e-16
std,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00
min,-5.163086e-01,-1.936826e+00,-1.085425e+00,-4.822104e-01,-5.315237e-01,-2.298105e-01,-0.507826,-1.226974e+00,-4.907810e-01,-1.213136e+00,-8.243101e-01,-2.716072e-01,-1.001091,-8.698879e-01,-2.704405e+00,-2.920572e+00,-2.994023e+00,-5.517333e-01,-3.016886e+00,-3.269662e+00
25%,-5.163086e-01,5.163086e-01,-1.085425e+00,-4.822104e-01,-5.315237e-01,-2.298105e-01,-0.507826,-1.226974e+00,-4.907810e-01,-1.213136e+00,-8.243101e-01,-2.716072e-01,-1.001091,-8.698879e-01,-6.900904e-01,-6.971063e-01,-5.745784e-01,-5.517333e-01,-6.596226e-01,-8.315022e-01
50%,-5.163086e-01,5.163086e-01,9.212982e-01,-4.822104e-01,-5.315237e-01,-2.298105e-01,-0.507826,8.150134e-01,-4.907810e-01,8.243101e-01,-8.243101e-01,-2.716072e-01,0.998910,-8.698879e-01,5.202558e-02,-1.412398e-01,3.198360e-15,-5.517333e-01,4.755658e-02,-2.688500e-01
75%,-5.163086e-01,5.163086e-01,9.212982e-01,-4.822104e-01,-5.315237e-01,-2.298105e-01,-0.507826,8.150134e-01,-4.907810e-01,8.243101e-01,1.213136e+00,-2.716072e-01,0.998910,1.149573e+00,6.881250e-01,4.146267e-01,4.194568e-01,-5.517333e-01,7.547357e-01,5.751284e-01
max,1.936826e+00,5.163086e-01,9.212982e-01,2.073784e+00,1.881384e+00,4.351412e+00,1.969177,8.150134e-01,2.037569e+00,8.243101e-01,1.213136e+00,3.681787e+00,0.998910,1.149573e+00,2.490407e+00,3.749826e+00,6.721265e+00,1.812470e+00,2.561971e+00,4.982571e+00


## Resumo pré-processamento

alvo = variável que se pretende atingir (tem ou nào oença cardíaca)
previsores = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numérica manualmente, sem escalonar
previsores_esc = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas, escalonada
previsores2 = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numérica pelo Labelencoder
previsores3 = conjunto de variáveis prevsoras transformadas pelo LabelEncoder e OneHotEncoder, sem escalonar
previsores3_esc = conjunto de variáveis previsoras transformadas LabelEncoder e OneHotEncoder, escalonada

## Redução de dimensionalidade

O objetivo é selecionar os melhores componentes (atributos), para treinamento do algoritmo, através da análise das correlações entre as variáveis. 
Útil em datasets com muitos atributos (colunas).

### Análise dos Componentes Principais (PCA)
**Seleção de características:** seleciona os melhores atributos e utiliza sem tranformações
**Extração de características:** encontra os relacionamentos dos melhores atributos e cria novos atributos
É um algoritmo de apredizagem não-supervisionada (trabalha sozinho)
Aplica-se em dados linearmente separáveis (que se consegue dividir conforme suas características)

In [86]:
from sklearn.decomposition import PCA

In [87]:
pca = PCA(n_components=4)

In [88]:
previsores_pca = pca.fit_transform(previsores2)

In [ ]:
previsores_pca.shape

(917, 4)

In [ ]:
# componentes principais
previsores_pca

array([[  44.01218603,   36.15137592,   10.64727631,   -9.47126222],
       [ -63.9918455 ,   13.93849233,   31.69212503,   -5.30052573],
       [  38.53844277,  -33.91712222,  -12.48837374,  -21.53860158],
       ...,
       [-113.3467467 ,  -23.48595247,   -2.51064362,    1.14935806],
       [  -9.1141306 ,   35.90083188,    4.81814643,    9.1238555 ],
       [ -70.01342608,   35.68741686,   12.10984845,  -10.51663588]],
      shape=(917, 4))

In [ ]:
# razão das variáveis explicativas
pca.explained_variance_ratio_

# result: array([0.72847929, 0.17183534, 0.08122019, 0.01767209])
# no caso, este componente consegue explicar 72,8% a primeira coluna e assim por diante nas demais colunas

array([0.72847929, 0.17183534, 0.08122019, 0.01767209])

In [92]:
# soma das variáveis explicativas
pca.explained_variance_ratio_.sum()

np.float64(0.9992069186607355)

### Kernel PCA

Algoritmo de aprendizagem não supervisionada<br />
Aplica-se também em dados linearmente não separáveis

In [ ]:
from sklearn.decomposition import KernelPCA

In [94]:
kpca = KernelPCA(n_components=4, kernel='rbf')

In [95]:
previsores_kernel = kpca.fit_transform(previsores2)

In [97]:
previsores_kernel.shape

(917, 4)

In [98]:
previsores_kernel

array([[-0.00249772, -0.00290225, -0.0027372 , -0.00191636],
       [-0.00249877, -0.00290359, -0.00273854, -0.00191743],
       [-0.00249785, -0.00290241, -0.00273736, -0.00191649],
       ...,
       [-0.00249774, -0.00290227, -0.00273723, -0.00191638],
       [-0.00249793, -0.00290252, -0.00273748, -0.00191659],
       [-0.00249772, -0.00290225, -0.0027372 , -0.00191637]],
      shape=(917, 4))

### Análise do Discriminante Linear (LDA: Linear Discriminant Analysis)

Algoritmo de aprendizagem supervisionada, pois usa a classe como referência para a seleção<br />
Aplicado em situações com muitos atributos previsores e também com o atributo alvo com muitas classes (no caso não serve porque o alvo é apenas 2, se tem ou não risco de doença cardíaca)

In [99]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=1)